# Getting Started with COMPAS FD

`compas_fd` is a package for computing equilibrium shapes of tensile surface structures consisting of axial force members connected by articulated nodes, using the force density method.

This notebook contains a very simple, step-by-step procedure for calculating the the shape of a hypar. The notebook can be run directly in the browser without any installation requirements.

For more information about `compas_fd`, please refer to the Github repo and the package documentation:

* [github repo](https://github.com/blockresearchgroup/compas_fd)
* [online docs](https://blockresearchgroup.github.io/compas_fd/latest/)

## Installation

The basic Jupyter infrastructure packages have already been pre-installed. However, COMPAS packages still have to be downloaded and installed "on-the-fly". Therefore, it is recommended that you run the cells one by one, instead of all cells at once. Especially the execution of the cell below should be completed before running any other cells. Otherwise, you will encounter errors related to the COMPAS packages not being installed (yet)...

In [1]:
%pip install -q jsonschema networkx numpy scipy pythreejs
%pip install -q compas>=2.8 --no-deps
%pip install -q compas_notebook>=0.10.0 --no-deps
%pip install -q compas_fd>=0.5 --no-deps

Note: you may need to restart the kernel to use updated packages.
zsh:1: 2.8 not found
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
zsh:1: 0.5 not found
Note: you may need to restart the kernel to use updated packages.


## Base Mesh

Create a mesh from a grid of vertices and faces in the XY plane.

In [2]:
from compas.datastructures import Mesh

mesh = Mesh.from_meshgrid(dx=10, nx=10)

Move two opposite mesh corners out of the XY plane.

In [3]:
vertex = list(mesh.vertices_where(x=0, y=0))[0]
mesh.vertex_attribute(vertex, name='z', value=7)

vertex = list(mesh.vertices_where(x=10, y=10))[0]
mesh.vertex_attribute(vertex, name='z', value=7)

## Numerical Data

Extract the vertex coordinates.

In [4]:
vertices = mesh.vertices_attributes("xyz")

Identify the corners as fixed points, which will be able to provide reaction forces.

In [5]:
fixed = list(mesh.vertices_where(vertex_degree=2))

Create a list of "zero" loads.

In [6]:
loads = [[0, 0, 0] for _ in range(len(vertices))]

Create a list of edges, with each edge defined as a pair of vertex indices.

In [7]:
edges = list(mesh.edges())

Assign force densities to the edges.
Use `q=10.0` for edges on the boundary, and `q=1.0` everywhere else.

In [8]:
q = []
for edge in edges:
    if mesh.is_edge_on_boundary(edge):
        q.append(10)
    else:
        q.append(1.0)

## Equilibrium

Compute equilibrium with the base force density solver of `compas_fd`.
The result is stored in a `Result` data class.

In [9]:
from compas_fd.solvers import fd_numpy

result = fd_numpy(
    vertices=vertices,
    fixed=fixed,
    edges=edges,
    forcedensities=q,
    loads=loads,
)

Update the original mesh geometry using the vertex coordinates stored in the result.

In [10]:
for vertex, attr in mesh.vertices(data=True):
    attr["x"] = result.vertices[vertex, 0]
    attr["y"] = result.vertices[vertex, 1]
    attr["z"] = result.vertices[vertex, 2]

## Visualisation

Visualize the equilibrium geometry and mark the fixed nodes.

In [11]:
from compas.colors import Color
from compas_notebook import Viewer
from compas_notebook.config import Config

config = Config()
config.view.camera.position = [15, -10, 3]
config.view.camera.target = [5, 5, 3]
config.view.camera.fov = 35

viewer = Viewer(config=config)

viewer.scene.add(mesh, color=Color.from_hex('#cccccc'), show_edges=True)

for vertex in fixed:
    viewer.scene.add(mesh.vertex_point(vertex), color=Color.red())

viewer.show()

# this is a temp hack until notebook is fixed
viewer.camera3.position = [15, -10, 3]
viewer.camera3.lookAt([5, 5, 3])
viewer.controls3.target = [5, 5, 3]